Problem Statement: Write a program to implment Parallel Bubble Sort. Use existing algorithms and measure the performance of sequectial and parallel algorithms. 

In [1]:
%%writefile Bubblesort.cpp
#include <iostream>
#include <chrono>
#include <omp.h>
#include <stdio.h>
#include <random>

using namespace std;
using namespace std::chrono;

int random_in_range( int minimum, int maximum )
{
  thread_local std::ranlux48 rng(
    std::chrono::system_clock::now().time_since_epoch().count() );
  return std::uniform_int_distribution <int> ( minimum, maximum )( rng );
}


void bubbleSortSerial(int a[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();

	for (int i = 0; i < n-1; i++)
	{
		for (int j = 0; j < n-1; j++)
		{
			if(a[j] > a[j+1])
			{
				int temp = a[j];
				a[j] = a[j+1];
				a[j+1] = temp;
			}
		}
	}
	endtime = system_clock::now();
	duration <double> time= endtime - starttime;
	cout<<"Time for serial: "<<1000*time.count()<<" milliseconds"<<endl;
}

void bubbleSortOddEven(int b[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	int pass;

	for(int i = 0 ; i < n-1 ; i++)
	{
		pass = i % 2;
		for (int j = pass ; j < n-1 ; j+=2)
		{
			if(b[j]>b[j+1])
			{
				int temp = b[j];
				b[j] = b[j+1];
				b[j+1]=temp;
			}
		}
	}
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for Bubble sort (Odd Even Transposition): "<<1000*time.count()<<" milliseconds"<<endl;
}

/*

void bubbleSortParallel(int b[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	int pass;

	omp_set_num_threads(2);

	for(int i = 0 ; i < n-1 ; i++)
	{
		pass = i % 2;
		#pragma omp parallel for
		for (int j = pass ; j < n-1 ; j+=2)
		{
			if(b[j]>b[j+1])
			{
				int temp = b[j];
				b[j] = b[j+1];
				b[j+1]=temp;
			}
		}
	}
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for Parallel: "<<1000*time.count()<<" milliseconds"<<endl;
}

*/

void init_array(int *arr1, int *arr2, int *arr3, int n) {
	int num;
	cout<<"Enter "<<n<<" elements :";
	for(int i = 0 ; i < n ; i++) {
		cin>>num;
		arr1[i] = arr2[i] = arr3[i] = num;
	}
}

void print_array(int *arr, int n) {
	for(int i = 0 ; i < n ; i++) {
		cout<<arr[i]<<" ";
	}
}

int main()
{
	int n;
	cout<<"Enter value of n :";
	cin>>n;
	int *a, *b, *c;
	a = new int[n];
	b = new int[n];
	c = new int[n];
	init_array(a, b, c, n);
	cout<<"Initial vector: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;

	bubbleSortSerial(a,n);
	cout<<"Result after serial bubble sort: "<<endl;
	print_array(a, n);

	cout<<endl;
	cout<<endl;

	bubbleSortOddEven(c,n);
	cout<<"Result after odd-even sort: "<<endl;
	print_array(c, n);

  /*
	cout<<endl;
	cout<<endl;
	bubbleSortParallel(b,n);
	cout<<"Result after parallel bubble sort: "<<endl;
	print_array(b, n);
	cout<<endl;
  */
	return 0;

}


Writing Bubblesort.cpp


In [2]:
!g++ -fopenmp Bubblesort.cpp -o Bubblesort_program

In [3]:
!./Bubblesort_program

Enter value of n :5
Enter 5 elements :3 2 7 5 9 
Initial vector: 
3 2 7 5 9 

Time for serial: 0.000319 milliseconds
Result after serial bubble sort: 
2 3 5 7 9 

Time for Bubble sort (Odd Even Transposition): 0.000216 milliseconds
Result after odd-even sort: 
2 3 5 7 9 

In [4]:
%%writefile Mergesort.cpp
#include <iostream>
#include <chrono>
#include <omp.h>
#include <stdio.h>
#include <random>
#include <string.h>
#include <stdlib.h>

using namespace std;
using namespace std::chrono;

int random_in_range( int minimum, int maximum )
{
  thread_local std::ranlux48 rng(
    std::chrono::system_clock::now().time_since_epoch().count() );
  return std::uniform_int_distribution <int> ( minimum, maximum )( rng );
}

void merge(int *c1, int *c2, int *p, int cl1, int cl2) {
	int i = 0, j = 0, k = 0;
	while(i < cl1 && j < cl2) {
		if(c1[i] < c2[j]) {
			p[k] = c1[i];
			i++;
			k++;
		}
		else {
			p[k] = c2[j];
			j++;
			k++;
		}
	}
	while(i < cl1) {
		p[k] = c1[i];
		i++;
		k++;
	}
	while(j < cl2) {
		p[k] = c2[j];
		j++;
		k++;
	}
}

void mergeSortSerial(int *arr, int length) {
	if(length <= 1)
		return;

	int mid = length / 2;

	int *left = new int[mid];
	int *right = new int[length-mid];

	for(int i = 0 ; i < mid ; i++) {
		left[i] = arr[i];
	}
	for(int i = mid ; i < length ; i++) {
		right[i-mid] = arr[i];
	}
	mergeSortSerial(left, mid);
	mergeSortSerial(right, length-mid);
	merge(left, right, arr, mid, length-mid);
}

void mergeSortParallel(int *arr, int length) {
	if(length <= 1)
		return;
	int mid = length / 2;
	int *left, *right;
	left = new int[mid];
	right = new int[length-mid];

	#pragma omp task firstprivate(left)
	{

		for(int i = 0 ; i < mid ; i++) {
			left[i] = arr[i];
		}
		mergeSortParallel(left, mid);
	}


	#pragma omp task firstprivate(right)
	{

		for(int i = mid ; i < length ; i++) {
			right[i-mid] = arr[i];
		}

		mergeSortParallel(right, length-mid);
	}


	// #pragma omp parallel sections
	// {
	// 	#pragma omp section
	// 	{
	// 		left = new int[mid];
	// 		for(int i = 0 ; i < mid ; i++) {
	// 			left[i] = arr[i];
	// 		}
	// 		mergeSortParallel(left, mid);
	// 	}
	// 	#pragma omp section
	// 	{
	// 		right = new int[length-mid];
	// 		for(int i = mid ; i < length ; i++) {
	// 			right[i-mid] = arr[i];
	// 		}
	// 		mergeSortParallel(right, length-mid);
	// 	}
	// }
	#pragma omp taskwait
	merge(left, right, arr, mid, length-mid);
}

void init_array(int *arr1, int *arr2, int n) {
	cout<<"Enter "<<n<<" elements :";
	int num;
	for(int i = 0 ; i < n ; i++) {
		cin>>num;
		arr1[i] = arr2[i] = num;
	}
}

void print_array(int *arr, int n) {
	for(int i = 0 ; i < n ; i++) {
		cout<<arr[i]<<" ";
	}
}

int main() {
	int n;
	cout<<"Enter the value of n :";
	cin>>n;
	int *a, *b;
	a = new int[n];
	b = new int[n];

	init_array(a, b, n);
	cout<<"Initial vector: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;

	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	mergeSortSerial(a, n);
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for serial: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after serial merge sort: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;


	starttime = system_clock::now();
	mergeSortParallel(b, n);
	endtime = system_clock::now();
	time = endtime - starttime;
	cout<<"Time for parallel: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after parallel merge sort: "<<endl;
	print_array(b, n);

	return 0;
}


Writing Mergesort.cpp


In [5]:
!g++ -fopenmp Mergesort.cpp -o Mergesort_program


In [6]:
!./Mergesort_program

Enter the value of n :5
Enter 5 elements :3 2 7 5 9 
Initial vector: 
3 2 7 5 9 

Time for serial: 0.003151 milliseconds
Result after serial merge sort: 
2 3 5 7 9 

Time for parallel: 0.002728 milliseconds
Result after parallel merge sort: 
2 3 5 7 9 